# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier 

%matplotlib inline
%load_ext pycodestyle_magic

In [4]:
# %%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        self.counter = dict()
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return (l_s - (np.sum(l_c * l_c / l_s, axis=1, keepdims=True)) + r_s - (
            np.sum(r_c * r_c / r_s, axis=1, keepdims=True)))
        # Ваш код в 1 строчку

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(np.sum((l_c * np.log2(((l_c / l_s).clip(min=1e-3, max=1 - 1e-3))) / l_s)
                        + (r_c * np.log2(((r_c / r_s).clip(min=1e-3, max=1 - 1e-3))) / r_s), axis=1, keepdims=True))
        # Ваш код в 1 строчку

    def __misclass(self, l_c, l_s, r_c, r_s):
        (l_s + r_s) - (np.max(l_c + r_c, axis=1, keepdims=True))
        # Ваш код в 1 строчку

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]  # Ваш код в 1 строчку

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]  # Ваш код в 1 строчку

    def __get_feature_ids_N(self, n_feature):
        return list(range(n_feature))  # Ваш код

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # Сортирует x и переставляет y по аргументам сортировки x
        # Записывает в class_number число уникальных значений из y
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]

        # Что делает этот блок кода?
        # берёт часть матрицы "откусывая" min_samples с двух сторон
        # ищем несовпадающие индексы
        # если таких нет, то всё принадлежит одному узлу 
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)

        if len(r_border_ids) == 0:
            return float('+inf'), None

        # Что делает этот блок кода?
        # Находим колличество элементов для каждого из кусочков
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split],
                                                                minlength=class_number)

        # Что делает этот блок кода?
        # подготавливает различные разбиения классов Sl Sr
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # Считает Gain для разных классов и выбирает min
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # Выбирает оценку соответ-ую min(gs) и считает treshold        
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id - 1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node

        count = np.bincount(y, minlength=self.num_class)
        prob = count / np.sum(count).astype(float)

        if self.max_depth is not None and depth == self.max_depth:
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(count), prob)
            return

        if self.min_samples_split is not None and x.shape[0] < self.min_samples_split:
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(count), prob)
            return

        gs = np.zeros(x.shape[1])
        thresholds = np.zeros(x.shape[1])

        first_label = np.min(y)
        y -= first_label

        for index in self.get_feature_ids(x.shape[1]):
            gs[index], thresholds[index] = self.__find_threshold(x[:, index], y)

        y += first_label
        first_group_index = np.argmin(gs)

        if np.isnan(thresholds[first_group_index]):
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(count), prob)
            return

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, first_group_index, thresholds[first_group_index])

        if y_l.shape[0] == 0 or y_r.shape[0] == 0:
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(count), prob)
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE, first_group_index, thresholds[first_group_index])

        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    def feature_importance(self):
        self.counter['count_node'] = 0
        self.feature_count(0)
        for key in self.counter:
            if key != 'count_node':
                self.counter[key] /= self.counter['count_node']
        return self.counter

    def feature_count(self, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            self.counter[node[1]] = self.counter[node[1]] + 1 if node[1] in self.counter else 0
            self.counter['count_node'] += 1
            self.feature_count(2 * node_id + 1)
            self.feature_count(2 * node_id + 2)


In [53]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [54]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [59]:
%time clf.fit(X_train, y_train)

CPU times: user 2.11 ms, sys: 0 ns, total: 2.11 ms
Wall time: 1.7 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [60]:
%time my_clf.fit(X_train, y_train)

CPU times: user 12.5 ms, sys: 0 ns, total: 12.5 ms
Wall time: 12.2 ms


## Проверка качества работы на wine

In [61]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8885003885003885

In [62]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8885003885003885

## Подготовка данных Speed Dating Data 

In [88]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv',
                 encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df.drop_duplicates(subset=['iid']).gender.value_counts()
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
             axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
pd.get_dummies(df, columns=['field_cd'],
               prefix='field_cd', prefix_sep='=')
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)
df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]

    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
    .drop(['gender'], axis=1) \
    .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
    .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
    .dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values


In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [95]:
%time clf.fit(X_train, y_train)

CPU times: user 100 ms, sys: 6 µs, total: 100 ms
Wall time: 99.3 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [96]:
%time my_clf.fit(X_train, y_train)

CPU times: user 570 ms, sys: 33 µs, total: 570 ms
Wall time: 569 ms


## Проверка скорости работы на Speed Dating Data 

In [97]:
# тут должен быть код типа 
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5188981490351354

In [98]:
# тут должен быть код типа 
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5251644024043263

## Проверка качества работы на Speed Dating Data

In [99]:
# тут должен быть код типа 
%time clf.fit(X_train, y_train)

CPU times: user 85.6 ms, sys: 30 µs, total: 85.6 ms
Wall time: 84.7 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [100]:
# тут должен быть код типа 
%time my_clf.fit(X_train, y_train)

CPU times: user 565 ms, sys: 0 ns, total: 565 ms
Wall time: 564 ms


## Задание 3

In [27]:
from sklearn.pipeline import Pipeline

df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv',
                 encoding='cp1251')


def clearn():
    global df
    global X, y
    df = df.iloc[:, :97]
    df = df.drop(['id'], axis=1)
    df = df.drop(['idg'], axis=1)
    df.drop_duplicates(subset=['iid']).gender.value_counts()
    df = df.drop(['condtn'], axis=1)
    df = df.drop(['round'], axis=1)
    df = df.drop(['position', 'positin1'], axis=1)
    df = df.drop(['order'], axis=1)
    df = df.drop(['partner'], axis=1)
    df = df.drop(['age_o', 'race_o', 'pf_o_att',
                  'pf_o_sin', 'pf_o_int',
                  'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
                  'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
                  'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
                 axis=1)
    df = df.dropna(subset=['age'])
    df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
    df = df.drop(['field'], axis=1)
    pd.get_dummies(df, columns=['field_cd'],
                   prefix='field_cd', prefix_sep='=')
    df = df.drop(['undergra'], axis=1)
    df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
    df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
    df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
    df.loc[:, 'tuition'] = df.tuition.fillna(-999)
    df = df.dropna(subset=['imprelig', 'imprace'])
    df = df.drop(['from', 'zipcode'], axis=1)
    df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
    df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
    df = df.dropna(subset=['date'])
    df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
    df = df.drop(['career'], axis=1)
    df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
                  'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga'], axis=1)
    df = df.drop(['expnum'], axis=1)
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(
        axis=1)
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
        (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T / df.loc[:,
                                                                                          'temp_totalsum'].T).T * 100
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(
        axis=1)
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
        (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T / df.loc[:,
                                                                                          'temp_totalsum'].T).T * 100
    df = df.drop(['temp_totalsum'], axis=1)
    for i in [4, 5]:
        feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
                'intel{}_1'.format(i), 'fun{}_1'.format(i),
                'amb{}_1'.format(i), 'shar{}_1'.format(i)]

        if i != 4:
            feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)
    df = df.drop(['wave'], axis=1)
    df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
        .drop(['gender'], axis=1) \
        .dropna()
    df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
        .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
        .dropna()

    df_female.columns = df_female.columns + '_f'
    df_female = df_female.drop(['pid_f'], axis=1)
    df_pair = df_male.join(df_female.set_index('iid_f'),
                           on='pid',
                           how='inner')
    df_pair = df_pair.drop(['iid', 'pid'], axis=1)
    X = df_pair.iloc[:, 1:].values
    y = df_pair.iloc[:, 0].values


def split():
    global X_train, X_test, y_train, y_test, X, y
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)


X = None
y = None
X_train = None
y_train = None
X_test = None
y_test = None
tred = Pipeline([('1', clearn()),
                 ('2', split()),
                 ('3', DecisionTreeClassifier(min_samples_split=2))])

tred.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('1', None), ('2', None), ('3', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

## Задание 4

In [101]:
s = my_clf.feature_importance()
s.pop('count_node')
print(X_test.shape[1])
buff = np.zeros(X_train.shape[1])
for i in s:
    buff[i] = s[i]

features_names = df_pair.columns[1:]
print(pd.Series(index=features_names, data=clf.feature_importances_) \
          .sort_values(ascending=False)[:10])
pd.Series(index=features_names, data=buff) \
    .sort_values(ascending=False)[:10]

62
int_corr      0.064535
intel1_1_f    0.031254
intel1_1      0.028266
fun2_1        0.028225
amb1_1_f      0.027720
fun1_1        0.026591
age_f         0.025712
amb2_1        0.025493
intel2_1      0.025402
fun3_1        0.022866
dtype: float64


mn_sat_f     0.056338
int_corr     0.028169
age          0.028169
tuition_f    0.028169
race         0.028169
intel1_1     0.028169
goal         0.014085
sinc3_1      0.014085
age_f        0.014085
income_f     0.014085
dtype: float64

## Задание 5

In [50]:
clf = RandomForestClassifier()
param_grid = {
    'n_estimators': [10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2']
}
CV_rfc = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_)

{'max_features': 'log2', 'n_estimators': 30}
